In [6]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import glob
import os
import pandas
class ImageMetaData(object):
    '''
    Extract the exif data from any image. Data includes GPS coordinates, 
    Focal Length, Manufacture, and more.
    '''
    exif_data = None
    image = None

    def __init__(self, img_path):
        self.image = Image.open(img_path)
        #print(self.image._getexif())
        self.get_exif_data()
        super(ImageMetaData, self).__init__()

    def get_exif_data(self):
        """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
        exif_data = {}
        info = self.image._getexif()
        if info:
            for tag, value in info.items():
                decoded = TAGS.get(tag, tag)
                if decoded == "GPSInfo":
                    gps_data = {}
                    for t in value:
                        sub_decoded = GPSTAGS.get(t, t)
                        gps_data[sub_decoded] = value[t]

                    exif_data[decoded] = gps_data
                else:
                    exif_data[decoded] = value
        self.exif_data = exif_data
        return exif_data

    def get_if_exist(self, data, key):
        if key in data:
            return data[key]
        return None

    def convert_to_degress(self, value):

        """Helper function to convert the GPS coordinates 
        stored in the EXIF to degress in float format"""
#         d0 = value[0][0]
#         d1 = value[0][1]
#         d = float(d0) / float(d1)

#         m0 = value[1][0]
#         m1 = value[1][1]
#         m = float(m0) / float(m1)

#         s0 = value[2][0]
#         s1 = value[2][1]
#         s = float(s0) / float(s1)
        d = value[0]
        m = value[1]
        s = value[2]

        return d + (m / 60.0) + (s / 3600.0)

    def get_lat_lng(self):
        """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
        lat = None
        lng = None
        exif_data = self.get_exif_data()
        #print(exif_data)
        if "GPSInfo" in exif_data:      
            gps_info = exif_data["GPSInfo"]
            gps_latitude = self.get_if_exist(gps_info, "GPSLatitude")
            gps_latitude_ref = self.get_if_exist(gps_info, 'GPSLatitudeRef')
            gps_longitude = self.get_if_exist(gps_info, 'GPSLongitude')
            gps_longitude_ref = self.get_if_exist(gps_info, 'GPSLongitudeRef')
            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = self.convert_to_degress(gps_latitude)
                if gps_latitude_ref != "N":                     
                    lat = 0 - lat
                lng = self.convert_to_degress(gps_longitude)
                if gps_longitude_ref != "E":
                    lng = 0 - lng
        return lat, lng



filenames=glob.glob('C:/Mayank/Projects/Project_CDRI/FieldPhotos/*.jpg')
imagename=[]
lat=[]
lon=[]
for filename in filenames:
    print(filename)
    path_name = filename
    meta_data =  ImageMetaData(path_name)
    latlng =meta_data.get_lat_lng()
    print(latlng)
    exif_data = meta_data.get_exif_data()
    lat.append(latlng[0])
    lon.append(latlng[1])
    imagename.append(os.path.basename(path_name))
#     print(exif_data)
print(len(filenames), len(lat), len(lon))



C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231009_20511pmByGPSMapCamera.jpg
(30.143226944444443, 78.59953388888889)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231009_20516pmByGPSMapCamera.jpg
(30.143226944444443, 78.59953388888889)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231011_114251AMByGPSMapCamera.jpg
(30.402280833333332, 79.04314083333333)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231011_114254AMByGPSMapCamera.jpg
(30.402280833333332, 79.04314083333333)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231011_114256AMByGPSMapCamera.jpg
(30.402280833333332, 79.04314083333333)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231011_114301AMByGPSMapCamera.jpg
(30.402280833333332, 79.04314083333333)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231011_114932amByGPSMapCamera.jpg
(30.40076583333333, 79.03784388888889)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\20231011_114933amByGPSMapCamera.jpg
(30.40076583333333, 79.03784388888889)
C:/Mayank/Projects/Project_CDRI/FieldPhotos\

In [7]:
df = pandas.DataFrame(data={"ImageName": imagename, "Lat": lat, "Long":lon})

In [8]:
df

,ImageName,Lat,Long
0,20231009_20511pmByGPSMapCamera.jpg,30.143227,78.599534
1,20231009_20516pmByGPSMapCamera.jpg,30.143227,78.599534
2,20231011_114251AMByGPSMapCamera.jpg,30.402281,79.043141
3,20231011_114254AMByGPSMapCamera.jpg,30.402281,79.043141
4,20231011_114256AMByGPSMapCamera.jpg,30.402281,79.043141
...,...,...,...
97,IMG_20231101_101641.jpg,29.651960,80.268881
98,IMG_20231101_101958.jpg,29.645734,80.274688
99,IMG_20231101_102000.jpg,29.645734,80.274688
100,IMG_20231101_102007.jpg,29.645734,80.274688


In [9]:
df.to_csv("C:/Mayank/Projects/Project_CDRI/FieldPhotos/Images_details_uttarakhandfield.csv", sep=',',index=False)